# Route Identification Over Time

Recent observations shows small chages in routes over time. Specifically in the following fields:
* route ID
* route short name
* route long name
* route desc

Need to observe these route changes in order to account for these changes in future analyses.

## Objective
1. Query data from `fct_monthly_routes` to help identify variences in Routes. Query for 2023, a couple of months. 
2. Save data to GCS `gtfs_schedule` bucket
3. Filter down data to `Sacramento Regional Transit`, identify and observe routes for any variences


In [1]:
#imports copied from download_vehicle_position.py script

import datetime
import gcsfs
import geopandas as gpd
import pandas as pd
import shapely
import sys

from calitp_data_analysis.tables import tbls
from calitp_data_analysis import utils
from loguru import logger
from siuba import *

from shared_utils import schedule_rt_utils
from shared_utils import geography_utils

/opt/conda/lib/python3.9/site-packages/google/auth/_default.py:78: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)
/opt/conda/lib/python3.9/site-packages/google/auth/_default.py:78: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)
/opt/conda/lib/python3.9/site-packages/google/auth/_default.py:78: UserWarning: Your application has authenticated using end user credentials from Googl

In [ ]:
# test to query fct_monthly_routes. DO NOT RUN

def get_monthly_routes(
        year: str,
        months: list
    ) -> pd.DataFrame:    
    
        df = (tbls.mart_gtfs.fct_monthly_routes()
              >> filter(_.year == year)
              >> filter(_.month.isin(months))
              >> select(_.key, _.source_record_id,
                        _.name,
                        _.route_id, _.shape_id,
                        _.month,
                        _.year,
                       _.pt_array)
                  >> collect()
             )
        return df

In [3]:
df = get_monthly_routes(2023, [4, 5, 6, 7, 8, 9])

/opt/conda/lib/python3.9/site-packages/sqlalchemy_bigquery/_types.py:101: SAWarning: Did not recognize type 'GEOGRAPHY' of column 'pt_array'
  sqlalchemy.util.warn(


In [6]:
#peaking into df to make sure everything looks good

#shape shows 11,927 rows and 8 columns
display(df.shape)

#type shows data is in df
display(type(df))

#columns return all the columns we listed in the function
display(list(df.columns))

#value_counts confirm df only has rows from 2023 March to May
display(df.value_counts(subset=['year','month']))

(24325, 8)

pandas.core.frame.DataFrame

['key',
 'source_record_id',
 'name',
 'route_id',
 'shape_id',
 'month',
 'year',
 'pt_array']

year  month
2023  8        4254
      6        4235
      5        4180
      9        4172
      4        3848
      7        3636
dtype: int64

### via Tiffany, make df to GPD
* <b>COMPLETE</b> Query data from warehouse
* then use this snippet from `make_routes_gdf` from `_shared_utils/shared_utils/geography_utils.py`.
    * `ddf["geometry"] = ddf.pt_array.apply(make_linestring)`
<br>  
* <b>COMPLETE</b> then save out as geo parquet to the `gtfs_schedule` folder in GCS (so versioning and history stays) using 
    * `utils.geoparquet_gcs_export(vp_gdf, SEGMENT_GCS, f"vp_{analysis_date}")`

In [8]:
#test of make_routes_gdf. DO NOT RUN

#aprl_sept_2023_routes = geography_utils.make_routes_gdf(df, "EPSG:4326")

In [9]:
#display(type(aprl_sept_2023_routes))
#list(aprl_sept_2023_routes.columns)
#display(aprl_sept_2023_routes.geometry.name)

geopandas.geodataframe.GeoDataFrame

'geometry'

In [ ]:
#writing to gcs bucket. DO NOT RUN AGAIN

#aprl_sept_2023_routes.to_parquet('gs://calitp-analytics-data/data-analyses/gtfs_schedule/route_identification_2023_m04_m09.parquet')


---

In [10]:
#creating sub-df for 'Sacramento Schedule'. DO NOT RUN
#195 rows, 8 columns
#sac = aprl_sept_2023_routes[aprl_sept_2023_routes['name'] == 'Sacramento Schedule']

In [ ]:
#writing sac filtered gdf to gcs. DO NOT RUN AGAIN

#sac.to_parquet('gs://calitp-analytics-data/data-analyses/gtfs_schedule/sac_route_identification_2023_m04_m09.parquet')

In [21]:
#Now we can read in the parquet file from gcs without having to remake everything again.

sac = pd.read_parquet('gs://calitp-analytics-data/data-analyses/gtfs_schedule/sac_route_identification_2023_m04_m09.parquet')

In [22]:
#test plot 
sac.geometry.iloc[1]


In [23]:
display(sac.shape)
display(sac.head())

(393, 8)

,key,source_record_id,name,route_id,shape_id,month,year,geometry
74,92ee64d95ea83c42a7922ae020226a66,recbzZQUIdMmFvm1r,Sacramento Schedule,213,45439,4,2023,b'\x01\x02\x00\x00\x00\xab\x00\x00\x00\xe4\xbc...
122,efb3865f306856498fc37d50203b2590,recbzZQUIdMmFvm1r,Sacramento Schedule,102,45972,8,2023,None
142,4def6187dada1c5406da712ebc6ffd53,recbzZQUIdMmFvm1r,Sacramento Schedule,019,45281,4,2023,b'\x01\x02\x00\x00\x00r\x02\x00\x00j\x88*\xfc\...
345,f73587c56d5d85c4a1b9eddd9c2a17c2,recbzZQUIdMmFvm1r,Sacramento Schedule,113,45985,8,2023,None
385,e3f865e8f09eb043485468f62a0957ba,recbzZQUIdMmFvm1r,Sacramento Schedule,056,45312,4,2023,b'\x01\x02\x00\x00\x00\xf1\x01\x00\x00\xf5g?RD...


In [24]:
sac_routes = sac['route_id'].unique()


In [25]:
len(sac_routes)
sac_routes

array(['213', '102', '019', '113', '056', '134', '015', '533', '138',
       '072', '011', 'F20', '246', '109', '061', '086', '062', '177',
       '205', '212', '087', '228', '106', '084', '038', '210', '252',
       '161', '227', '033', '248', '206', '021', '30', '124', '088',
       '142', '507', '255', '025', '103', '030', '175', '013', '075',
       'F10', '214', '067', '211', '215', '082', '247', '226', '129',
       '519', '078', '051', '093', '068', '023', '10', '026', '176',
       '137', '081', '001', '105'], dtype=object)

At this point determined we need more data that what `fct_monthly_routes` has. See approach 2

# HALL OF SHAME

---


## Next Steps

for every `name` and `route_id` in routes, need to see if each row is the same or not. Need to identify any variation in the routes. 



---

Trying to use a loop that will create a df for every route_id in sac_routes. but would need to do this for every `name` in the `fct_monthly_routes` df eventually?


In [ ]:
#empty dictionary
sac_sub_route_ids = {}

#each element in sac_routes will be called route.
#for each route in sac_routes, query each row related to that route.(where ever you use the variable route, go 1-by-1 the differnet
#then, create a dataframe for each route and place it into the dictionary sub_dataframes
for route in sac_routes:
    sub_df = sac[sac['route_id'] == route]
    sac_sub_route_ids[route] = sub_df

In [ ]:
len(sac_sub_route_ids)

In [ ]:
#testing dictionary with route 23 and 88
sac_sub_route_ids['023']

---

In [ ]:
#list of unique route names from initial df
route_names = df['name'].unique()


In [ ]:
#new loop that creates a dictionary of each unique schedule name with all its routes.
sub_route_name = {}

for name in route_names:
    sub_df2 = df[df['name'] == name]
    sub_route_name[name] = sub_df2
    

In [ ]:
#test to see if new dictionary works
sub_route_name['Auburn Schedule']

## Now I have `sac_sub_route_ids` and `sub_route_name` dictionaries

Examples of some noteable routes with slight variations over time.

In [ ]:
#General observations for Sacramento Schedule: 
#shape_id changes every month. 
#pt_array changes every month, however, did get a warning upon initial query of data so may need to review query to account for geodata
#Month 4 has the point geom data

display(sac_sub_route_ids['088'])
display(sac_sub_route_ids['023'])
display(sac_sub_route_ids['105'])
display(sac_sub_route_ids['F20'])
display(sac_sub_route_ids['215'])

In [ ]:
#test to see other route names
#other route names have more point geometry than Sacramento.
display(sub_route_name['Santa Cruz Schedule'])
display(sub_route_name['Merced Schedule'])
display(sub_route_name['San Diego Schedule'])
display(sub_route_name['Roseville Schedule'])

In [ ]:
import importlib

importlib.reload(segment_speed_utils)
from segment_speed_utils.project_vars import SCHED_GCS, analysis_date